In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

ModuleNotFoundError: No module named 'xgboost'

In [3]:
df = pd.read_csv('data/stud.csv')

In [7]:
df['total score'] = df['math_score'] + df['reading_score'] + df['writing_score']
df.head()


,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total score
0,female,group B,bachelor's degree,standard,none,72.0,72,74,218.0
1,female,group C,some college,standard,completed,69.0,90,88,247.0
2,female,group B,master's degree,standard,none,90.0,95,93,278.0
3,male,group A,associate's degree,free/reduced,none,47.0,57,44,148.0
4,male,group C,some college,standard,none,76.0,78,75,229.0


In [27]:
#Total_Score Is The Dependent(output) feature and rest are the independent feature
#Seperating Dependent And independent features
X = df.drop("total score" , axis=1) 
Y = df["total score"]
X.head(20)
X['parental_level_of_education'].unique()

array(["bachelor's degree", 'some college', "master's degree",
       "associate's degree", 'high school', 'some high school'],
      dtype=object)

In [39]:
#Here We Transfored The Data Such That We Are applying encoding on the categorical features and standardization of numerical features
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

categorical_features = [feature for feature in df.columns if df[feature].dtype == 'object']
numerical_features = [feature for feature in df.columns if df[feature].dtype != 'object']


transform = ColumnTransformer(
    transformers=[
        (
            'tnf1',OrdinalEncoder(categories=[
                    ["some high school","high school","associate's degree", "some college", "master's degree", "bachelor's degree"],
                    ["female", "male"]
                ]),["parental_level_of_education", "gender"]
        ),
        (
            'tnf2',
            OneHotEncoder(drop="first"),["race_ethnicity", "lunch", "test_preparation_course"]
        ),
        (
            ("scaler", StandardScaler(), numerical_features)

        )
    ],remainder="passthrough")

transformed_data = transform.fit_transform(df)
feature_names = transform.get_feature_names_out()
transformed_df = pd.DataFrame(transformed_data, columns=feature_names)

In [40]:
transformed_df

,tnf1__parental_level_of_education,tnf1__gender,tnf2__race_ethnicity_group B,tnf2__race_ethnicity_group C,tnf2__race_ethnicity_group D,tnf2__race_ethnicity_group E,tnf2__lunch_standard,tnf2__lunch_nan,tnf2__test_preparation_course_none,scaler__math_score,scaler__reading_score,scaler__writing_score,scaler__total score
0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.385649,0.191309,0.389824,0.341754
1,3.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.187675,1.425793,1.312856,1.020979
2,4.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.573493,1.768706,1.642511,1.747047
3,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.264134,-0.837428,-1.588103,-1.297754
4,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.649614,0.602804,0.455755,0.599391
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.441510,2.043036,1.774372,1.840733
1003,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-0.274264,-0.974593,-0.862863,-0.735637
1004,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.472238,0.122727,-0.203554,-0.196941
1005,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.121684,0.602804,0.587616,0.458862


In [41]:
#Train And Test Data Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transformed_df,Y,test_size=0.2,random_state=42)